In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"

from keras.models import Sequential, load_model
from keras.layers import *
from keras import optimizers
from keras import utils
from keras.callbacks import TensorBoard, ModelCheckpoint, ReduceLROnPlateau
import keras

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import math

import tensorflow as tf

# Importing matplotlib to plot images.
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

# Importing SK-learn to calculate precision and recall
import sklearn
from sklearn import metrics
from sklearn.model_selection import train_test_split, cross_val_score, LeaveOneGroupOut
from sklearn.utils import shuffle 

# Used for graph export
from tensorflow.python.framework import graph_util
from tensorflow.python.framework import graph_io
from keras import backend as K

import pickle as pkl
import h5py

from pathlib import Path
import os.path
import sys
import datetime
import time



Using TensorFlow backend.


In [2]:
import telegram
from keras.callbacks import Callback
from callbacks import TelegramCallback
from callbacks.TelegramData import TelegramData


# create callback
config = {
    'token': TelegramData.Token,   # paste your bot token
    'telegram_id': TelegramData.ID,                                   # paste your telegram_id
    'model_name': 'Jan_LSTM'
}

tg_callback = TelegramCallback(config)

In [3]:
class LoggingTensorBoard(TensorBoard):    

    def __init__(self, log_dir, settings_str_to_log, **kwargs):
        super(LoggingTensorBoard, self).__init__(log_dir, **kwargs)

        self.settings_str = settings_str_to_log

    def on_train_begin(self, logs=None):
        TensorBoard.on_train_begin(self, logs=logs)

        tensor =  tf.convert_to_tensor(self.settings_str)
        summary = tf.summary.text ("Run_Settings", tensor)

        with  tf.Session() as sess:
            s = sess.run(summary)
            self.writer.add_summary(s)

In [4]:
dfAll = pd.read_pickle("PklData/df_lstm_norm50.pkl")

lst = dfAll.userID.unique()
np.random.seed(42)
np.random.shuffle(lst)
test_ids = lst[-5:]
train_ids = lst[:-5]
print(train_ids, test_ids)

[ 1  2  9  6  4 14 17 16 12  3 10 18  5] [13  8 11 15  7]


In [5]:
dfAll.userID.unique()

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18])

In [6]:
dfAll.TaskID = dfAll.TaskID % 17


In [7]:
df_train = dfAll[dfAll.userID.isin(train_ids)]
df_test = dfAll[dfAll.userID.isin(test_ids)]

df_train2 = df_train[['Blobs', 'TaskID']].copy()
df_test2 = df_test[['Blobs', 'TaskID']].copy()

In [8]:
x_train = np.concatenate(df_train2.Blobs.values).reshape(-1,50,27,15,1)
x_test = np.concatenate(df_test2.Blobs.values).reshape(-1,50,27,15,1)

y_train = df_train2.TaskID.values
y_test = df_test2.TaskID.values

x_train = x_train / 255.0
x_test = x_test / 255.0

In [9]:
# convert class vectors to binary class matrices (one-hot notation)
num_classes = len(dfAll.TaskID.unique())
y_train_one_hot = utils.to_categorical(df_train2.TaskID, num_classes)
y_test_one_hot = utils.to_categorical(df_test2.TaskID, num_classes)

In [10]:
# If GPU is not available: 
# GPU_USE = '/cpu:0'
#config = tf.ConfigProto(device_count = {"GPU": 1})


# If GPU is available: 
config = tf.ConfigProto()
config.log_device_placement = True
config.allow_soft_placement = True
config.gpu_options.allow_growth=True
config.gpu_options.allocator_type = 'BFC'

# Limit the maximum memory used
config.gpu_options.per_process_gpu_memory_fraction = 0.4

# set session config
tf.keras.backend.set_session(tf.Session(config=config))

In [ ]:
batch_size = 50
epochs = 3000
timesteps = 50
data_dim = (27,15)
l1v = 0.007
l2v = 0.014


tf.get_default_graph()
model = Sequential()

model.add(TimeDistributed(Conv2D(64, kernel_size=(3,3), activation='relu', 
            padding='same', kernel_regularizer=regularizers.l1_l2(l1v,l2v)), input_shape=(timesteps ,27, 15, 1)))
model.add(TimeDistributed(Conv2D(32, kernel_size=(3,3), activation='relu', padding='same', 
                                 kernel_regularizer=regularizers.l1_l2(l1v,l2v))))
model.add(TimeDistributed(MaxPooling2D(pool_size=(2,2), strides=None, padding='same', data_format='channels_last')))
model.add(TimeDistributed(Dropout(0.50)))

model.add(TimeDistributed(Conv2D(32, kernel_size=(3, 3), activation='relu', 
            padding='same', kernel_regularizer=regularizers.l1_l2(l1v,l2v)), input_shape=(timesteps ,27, 15, 1)))
model.add(TimeDistributed(Conv2D(16, kernel_size=(3, 3), activation='relu', padding='same', 
                                 kernel_regularizer=regularizers.l1_l2(l1v,l2v))))
model.add(TimeDistributed(MaxPooling2D(pool_size=(2,2), strides=None, padding='same', data_format='channels_last')))
model.add(TimeDistributed(Dropout(0.50)))

model.add(TimeDistributed(Flatten()))
#model.add(TimeDistributed(Dense(128)))
model.add(TimeDistributed(Dense(64)))
#model.add(TimeDistributed(Dense(32)))
model.add(LSTM(256, return_sequences=True, input_shape=(timesteps, data_dim),kernel_regularizer=regularizers.l1_l2(l1v,l2v)))
#model.add(Dropout(0.25))
#model.add(LSTM(256, return_sequences=True, input_shape=(timesteps, data_dim),kernel_regularizer=regularizers.l1_l2(0.001,0.01)))
#model.add(Dense(512))
model.add(LSTM(64, return_sequences=False, input_shape=(timesteps, data_dim),kernel_regularizer=regularizers.l1_l2(l1v,l2v)))
#model.add(Dropout(0.20))
#model.add(Dense(256))
#model.add(LSTM(64, kernel_regularizer=regularizers.l2(0.01)))
#model.add(Dropout(0.20))
#model.add(Dense(64))
model.add(Dense(num_classes, activation='softmax'))

#optimizer = optimizers.Adagrad()
optimizer = optimizers.Adam(lr = 0.0001, decay=1e-6)
#optimizer = optimizers.RMSprop(lr=0.001, rho=0.9, epsilon=1e-08, decay=0.1)
model.compile(loss='categorical_crossentropy',
                  optimizer=optimizer,
                  metrics=['accuracy'])

# Broadcast progress to the tensorboard.
readable_timestamp = datetime.datetime.fromtimestamp(time.time()).strftime('%Y%m%d_%H%M%S')
tensorflowfolder = "/srv/share/tensorboardfiles/KnuckleFinger_LSTM_Jan_" + readable_timestamp

config = ""
for layer in model.layers:
    config += str(layer.output).split('\"')[1].split("/")[0] + str(layer.output_shape) + "\n\n"
config += "batchsize: " + str(batch_size) + "\n\n" + "epochs: " + str(epochs) + "\n\n" 
config += "l1: " + str(l1v) + "\n\n" + "l2: " + str(l2v) + "\n\n"

model.summary()

readable_timestamp = datetime.datetime.fromtimestamp(time.time()).strftime('%Y%m%d_%H%M%S')
tensorflowfolder = "/srv/share/tensorboardfiles/KnuckleFinger_LSTM_Jan_" + readable_timestamp
print("KnuckleFinger_LSTM_Jan_" + readable_timestamp)
logger = LoggingTensorBoard(settings_str_to_log = config, log_dir=tensorflowfolder, histogram_freq=0,
                            write_graph=True, write_images=True, update_freq = 'epoch')
storer = ModelCheckpoint("./ModelSnapshots/KnuckleFinger_LSTM_Jan_" + readable_timestamp + ".h5", monitor='val_loss', verbose=0,
                         save_best_only=True, save_weights_only=False, mode='auto', period=1)
learning_rate_reduction = ReduceLROnPlateau(monitor='val_loss', 
                                            patience=100, 
                                            verbose=1, 
                                            factor=0.95, 
                                            min_lr=0.00001)

history = model.fit(x_train, y_train_one_hot,
                        batch_size=batch_size,
                        epochs=epochs,
                        verbose=1,
                        validation_data=(x_test, y_test_one_hot),
                        callbacks=[storer,logger,tg_callback, learning_rate_reduction])


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed_11 (TimeDis (None, 50, 27, 15, 64)    640       
_________________________________________________________________
time_distributed_12 (TimeDis (None, 50, 27, 15, 32)    18464     
_________________________________________________________________
time_distributed_13 (TimeDis (None, 50, 14, 8, 32)     0         
_________________________________________________________________
time_distributed_14 (TimeDis (None, 50, 14, 8, 32)     0         
_________________________________________________________________
time_distributed_15 (TimeDis (None, 50, 14, 8, 32)     9248      
_________________________________________________________________
time_distributed_16 (TimeDis (None, 50, 14, 8, 16)     4624      
_________________________________________________________________
time_distributed_17 (TimeDis (None, 50, 7, 4, 16)      0         
__________

6624/6624 [==============================] - 99s 15ms/step - loss: 2.3447 - acc: 0.2503 - val_loss: 2.4776 - val_acc: 0.2456
Epoch 45/3000
6624/6624 [==============================] - 99s 15ms/step - loss: 2.3405 - acc: 0.2497 - val_loss: 2.4734 - val_acc: 0.2374
Epoch 46/3000
6624/6624 [==============================] - 99s 15ms/step - loss: 2.3359 - acc: 0.2554 - val_loss: 2.4090 - val_acc: 0.2573
Epoch 47/3000
6624/6624 [==============================] - 100s 15ms/step - loss: 2.3640 - acc: 0.2506 - val_loss: 2.5337 - val_acc: 0.2184
Epoch 48/3000
6624/6624 [==============================] - 100s 15ms/step - loss: 2.3373 - acc: 0.2497 - val_loss: 2.4915 - val_acc: 0.2390
Epoch 49/3000
6624/6624 [==============================] - 98s 15ms/step - loss: 2.3313 - acc: 0.2539 - val_loss: 2.3882 - val_acc: 0.2585
Epoch 50/3000
6624/6624 [==============================] - 99s 15ms/step - loss: 2.3222 - acc: 0.2601 - val_loss: 2.4227 - val_acc: 0.2581
Epoch 51/3000
6624/6624 [==============

Epoch 103/3000
6624/6624 [==============================] - 96s 14ms/step - loss: 2.1081 - acc: 0.3220 - val_loss: 2.1771 - val_acc: 0.3196
Epoch 104/3000
6624/6624 [==============================] - 97s 15ms/step - loss: 2.1170 - acc: 0.3130 - val_loss: 2.1686 - val_acc: 0.3340
Epoch 105/3000
6624/6624 [==============================] - 96s 15ms/step - loss: 2.1155 - acc: 0.3169 - val_loss: 2.1824 - val_acc: 0.3215
Epoch 106/3000
6624/6624 [==============================] - 96s 15ms/step - loss: 2.1042 - acc: 0.3196 - val_loss: 2.2400 - val_acc: 0.2954
Epoch 107/3000
6624/6624 [==============================] - 96s 14ms/step - loss: 2.1114 - acc: 0.3197 - val_loss: 2.1742 - val_acc: 0.3184
Epoch 108/3000
6624/6624 [==============================] - 97s 15ms/step - loss: 2.0958 - acc: 0.3149 - val_loss: 2.1793 - val_acc: 0.3196
Epoch 109/3000
6624/6624 [==============================] - 97s 15ms/step - loss: 2.1025 - acc: 0.3204 - val_loss: 2.2142 - val_acc: 0.3025
Epoch 110/3000
6624/

In [ ]:
import matplotlib.pyplot as plt
plt.plot(history.history['val_acc'], label="Test Accuracy")
plt.plot(history.history['acc'], label="Training Accuracy")
plt.legend()


In [ ]:
print("end")

In [ ]:
model.save('lstm_01_03_19_w50_b40.h5')

In [ ]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    

In [ ]:

y_test_pred = model.predict(x_test, batch_size=30)
y_test_pred = np.argmax(y_test_pred, axis=1)
#model.predict(x_test)

print ('\n Summary of the precision, recall, F1 score for each class:')
print (sklearn.metrics.classification_report(y_test, y_test_pred))

print ('\n Confusion matrix: ')
print (sklearn.metrics.confusion_matrix(y_test, y_test_pred))